In [1]:
import numpy as np

from two_step_classification import *
print_version()

python:      3.10.11
sklearn:     1.2.2
tensorflow:  2.12.0
keras:       2.12.0
numpy:       1.23.5
pandas:      1.5.3


In [2]:
# load data
x_test: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_test-drop_25_df.pkl")
y_test: pd.Series = pd.read_pickle("models/kdd99_features/y_test_df.pkl")
y_test_binary: pd.Series = y_test.apply(lambda x: 0 if x == 1 else 1)
y_test_anomaly: pd.Series = pd.read_pickle("models/kdd99_features/y_test_dropped_mapped_series.pkl")
x_test_ae_35: pd.DataFrame = pd.read_pickle("models/kdd99_features/x_test-drop+ae_35_df&activation=relu&epochs=5&batch_size=32.pkl")

In [3]:
# load models
with open("models/logistic_regression_binary/kdd99-drop_25&penalty=l1&solver=liblinear&C=0.1.pkl", 'rb') as fp:
    model_1st: LogisticRegression = pickle.load(fp)
with open("models/logistic_regression_anomaly/kdd99-dropped_mapped&penalty=l1&solver=liblinear&C=1.pkl", 'rb') as fp:
    model_2nd: LogisticRegression = pickle.load(fp)

In [4]:
# 第1段階: 正常と異常の2値分類
y_pred_binary:pd.Series = classification_normal_and_anomaly(x_test, model_1st)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index
y_pred_binary.value_counts()

1    129863
0     33164
dtype: int64

In [5]:
y_pred_normal: pd.Series = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
y_pred_normal.value_counts()

1    33164
dtype: int64

In [6]:
cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])

In [7]:
# 第二段階
x_anomalies: pd.DataFrame = x_test.loc[predicted_indexes]
y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
wrapper = {0: 0, 1: 2, 2: 3, 3: 4}
y_pred_anomalies = y_pred_anomalies.apply(lambda x: wrapper[x])
y_pred_anomalies.value_counts()

0    128719
2       929
3       197
4        18
dtype: int64

In [8]:
y_pred = pd.concat([y_pred_normal, y_pred_anomalies])
cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())

In [9]:
y_pred.value_counts()

0    128719
1     33164
2       929
3       197
4        18
dtype: int64

In [10]:
cm_1st

,normal,anomaly
true_normal,31666,436
true_anomaly,1498,129427


In [11]:
cm_2nd

,dos,normal,probe,r2l,u2r
true_dos,128429,750,0,2,0
true_normal,285,31666,36,106,9
true_probe,2,459,892,1,1
true_r2l,3,280,1,88,0
true_u2r,0,9,0,0,8


In [12]:
# TODO: ここから先が完成していない．

model_1st: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_dropped_binary_tuned_booster.model")
model_2nd: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_dropped_mapped_anomaly_tuned_booster.model")
y_pred_binary = classification_normal_and_anomaly(x_test, model_1st)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index
x_anomalies = x_test.loc[predicted_indexes]
y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
print(y_pred_anomalies.value_counts())
y_pred_normal = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
y_pred = pd.concat([y_pred_normal, y_pred_anomalies])

y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)

cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])

cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index())

0    129631
1       761
2       380
3       144
dtype: int64


In [13]:
pd.Series(np.argmax(model_2nd.predict(x_train), axis=1)).value_counts()

NameError: name 'x_train' is not defined

In [ ]:
model_1st: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_dropped_binary_tuned_booster.model")
model_2nd: lgb.Booster = lgb.Booster(model_file="models/lightgbm/lgb_dropped_mapped_anomaly_tuned_booster.model")
y_pred_binary = classification_normal_and_anomaly(x_test, model_1st)
predicted_indexes = y_pred_binary[y_pred_binary == 1].index
x_anomalies = x_test.loc[predicted_indexes]
y_pred_anomalies: pd.Series = classification_anomalies(x_anomalies, model_2nd)
print(y_pred_anomalies.value_counts())
y_pred_normal = y_pred_binary[y_pred_binary == 0].apply(lambda _: 1)
y_pred = pd.concat([y_pred_normal, y_pred_anomalies])

y_test_binary = y_test.apply(lambda x: 0 if x == 1 else 1)

cm_1st = confusion_matrix_df(y_test_binary.sort_index(), y_pred_binary.sort_index(), labels=['normal', 'anomaly'])

cm_2nd = confusion_matrix_df(y_test.sort_index(), y_pred.sort_index(), labels=correspondences_anomaly.keys())

In [ ]:
cm_1st

In [ ]:
cm_2nd

In [ ]:
confusion_matrix(y_test.sort_index(), y_pred.sort_index())